In [ ]:
pip install pyspark spark-nlp pandas matplotlib requests

In [ ]:
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, udf, lower, trim, regexp_replace, collect_list
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT
import os

# Cài đặt Spark NLP
import sparknlp
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, XlmRoBertaEmbeddings

# Khởi tạo SparkSession kết nối đến cluster
spark = SparkSession.builder \
    .appName("Spark NLP Clustering") \
    .master("spark://172.18.0.3:7077") \
    .config("spark.driver.memory", "8g") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.0") \
    .getOrCreate()

api_key = ""

# URL của API Gemini 2.5 (cần thay thế bằng endpoint chính xác của Gemini 2.5)
url = "https://gemini.googleapis.com/v1/generate"

# Đọc file JSONL
input_file_path = "/opt/workspace/gen_1604_formated.jsonl"
df = spark.read.option("multiLine", False).json(input_file_path)

# Trích xuất các câu hỏi từ user
user_questions = df.select(explode("messages").alias("msg")) \
    .filter(col("msg.role") == "user") \
    .select(col("msg.content").alias("text")) \
    .filter(col("text").isNotNull())

# Kiểm tra số lượng câu hỏi
print(f"Số câu hỏi từ người dùng: {user_questions.count()}")

# NLP Pipeline
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
embeddings = XlmRoBertaEmbeddings.pretrained("xlmroberta_embeddings_afriberta_base", "xx") \
    .setInputCols(["document", "token"]).setOutputCol("embeddings")

pipeline = Pipeline(stages=[document_assembler, tokenizer, embeddings])
model = pipeline.fit(user_questions)
embedded_data = model.transform(user_questions)

# Tính trung bình vector
def avg_embeddings(embeddings):
    if embeddings:
        avg = np.mean([e['embeddings'] for e in embeddings], axis=0)
        return Vectors.dense(avg.tolist())
    return Vectors.dense([])

avg_embeddings_udf = udf(avg_embeddings, VectorUDT())
vectorized_data = embedded_data.withColumn("features", avg_embeddings_udf(col("embeddings")))

# Chuẩn hóa vector
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(vectorized_data)
scaled_data = scaler_model.transform(vectorized_data)

# KMeans clustering
k = 5  # Số chủ đề bạn muốn chia, có thể điều chỉnh
kmeans = KMeans(featuresCol="scaled_features", predictionCol="cluster", k=k)
kmeans_model = kmeans.fit(scaled_data)
clustered_data = kmeans_model.transform(scaled_data)

# Kiểm tra số lượng câu hỏi trong mỗi cluster
cluster_count = clustered_data.groupBy("cluster").count().orderBy("cluster")
cluster_count.show()

# Gom nhóm các câu hỏi theo cluster
topics_df = clustered_data.groupBy("cluster") \
    .agg(collect_list("text").alias("questions")) \
    .orderBy("cluster")

topics = topics_df.collect()

# Hàm gán tên chủ đề cho mỗi cụm bằng cách sử dụng LLM
def generate_topic_name_with_gemini(questions):
    prompt = "Dưới đây là một số câu hỏi của người dùng, hãy tạo ra một tên chủ đề ngắn gọn và ngữ nghĩa cho chúng:\n"
    prompt += "\n".join(questions)  # Thêm các câu hỏi vào prompt

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "gemini-2.5",  # Chỉ định mô hình Gemini 2.5
        "prompt": prompt,
        "max_tokens": 50,  # Giới hạn số lượng token cho tên chủ đề
        "temperature": 1.0,  # Độ sáng tạo của kết quả
    }

    # Gửi yêu cầu đến API Gemini
    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        result = response.json()
        return result['text'].strip()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return "Không thể tạo tên chủ đề"

# Gán tên chủ đề và gom lại
results = []
for row in topics:
    cluster_id = row["cluster"]
    questions = row["questions"]
    topic_name = generate_topic_name_with_gemini(questions)
    
    # Đảm bảo không trùng tên
    while topic_name in [r[0] for r in results]:
        topic_name += "_"
    
    results.append([topic_name, questions])

# Kiểm tra tên chủ đề và câu hỏi
for topic in results:
    print(f"Chủ đề: {topic[0]}")
    print(f"Các câu hỏi: {topic[1][:5]}")  # Hiển thị 5 câu hỏi đầu tiên trong chủ đề
    print("-" * 50)

# Chuyển kết quả thành DataFrame và lưu vào file CSV
df_results = pd.DataFrame(results, columns=["Topic Name", "Questions"])
df_results.to_csv("/opt/workspace/clustered_topics.csv", index=False, encoding="utf-8")

print(f"✅ Đã phân cụm và lưu kết quả vào: /opt/workspace/clustered_topics.csv")

# Kiểm tra số lượng chủ đề và câu hỏi trong kết quả
print(f"Số lượng chủ đề trong kết quả: {len(results)}")
print(f"Số câu hỏi trong kết quả: {df_results.shape[0]}")
